# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(22)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=50  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 
model = mobilenet(num_classes=10, large_img=False)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [4]:
# Implement validation
def train(epoch):
    model.train()
    writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            writer.add_scalar('Loss/Loss', loss.item(), epoch)
            writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [5]:
def validate(epoch):
    model.eval()
    writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [6]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [7]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [8]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
writer = SummaryWriter()
writer.export_scalars_to_json("./all_scalars.json")

writer.close()

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.304640, Accuracy: 7.81
Train Epoch: 0 [2560/50000 (6%)]	Loss: 2.375479, Accuracy: 13.67
Train Epoch: 0 [5120/50000 (11%)]	Loss: 1.988270, Accuracy: 25.78
Train Epoch: 0 [7680/50000 (17%)]	Loss: 2.038945, Accuracy: 27.34
Train Epoch: 0 [10240/50000 (23%)]	Loss: 1.903131, Accuracy: 26.56
Train Epoch: 0 [12800/50000 (28%)]	Loss: 1.943959, Accuracy: 28.52
Train Epoch: 0 [15360/50000 (34%)]	Loss: 2.084677, Accuracy: 24.22
Train Epoch: 0 [17920/50000 (40%)]	Loss: 1.904895, Accuracy: 26.56
Train Epoch: 0 [20480/50000 (45%)]	Loss: 1.815676, Accuracy: 29.69
Train Epoch: 0 [23040/50000 (51%)]	Loss: 1.820209, Accuracy: 31.64
Train Epoch: 0 [25600/50000 (57%)]	Loss: 1.765703, Accuracy: 32.03
Train Epoch: 0 [28160/50000 (62%)]	Loss: 1.795561, Accuracy: 32.81
Train Epoch: 0 [30720/50000 (68%)]	Loss: 1.691612, Accuracy: 38.67
Train Epoch: 0 [33280/50000 (74%)]	Loss: 1.778997, Accuracy: 35.55
Train Epoch: 0 [35840/50000 (80%)]	Loss: 1.712279, Accuracy: 38.67
Train

Train Epoch: 6 [12800/50000 (28%)]	Loss: 0.879068, Accuracy: 67.58
Train Epoch: 6 [15360/50000 (34%)]	Loss: 0.871549, Accuracy: 68.36
Train Epoch: 6 [17920/50000 (40%)]	Loss: 0.957298, Accuracy: 67.19
Train Epoch: 6 [20480/50000 (45%)]	Loss: 0.890900, Accuracy: 69.92
Train Epoch: 6 [23040/50000 (51%)]	Loss: 0.914453, Accuracy: 68.75
Train Epoch: 6 [25600/50000 (57%)]	Loss: 0.768623, Accuracy: 73.05
Train Epoch: 6 [28160/50000 (62%)]	Loss: 0.793022, Accuracy: 73.05
Train Epoch: 6 [30720/50000 (68%)]	Loss: 1.031054, Accuracy: 60.94
Train Epoch: 6 [33280/50000 (74%)]	Loss: 1.017033, Accuracy: 62.50
Train Epoch: 6 [35840/50000 (80%)]	Loss: 0.963585, Accuracy: 70.31
Train Epoch: 6 [38400/50000 (85%)]	Loss: 0.841889, Accuracy: 71.48
Train Epoch: 6 [40960/50000 (91%)]	Loss: 0.723412, Accuracy: 73.83
Train Epoch: 6 [43520/50000 (97%)]	Loss: 0.898974, Accuracy: 66.80

Validation set: Average loss: 1.1921, Accuracy: 3115/5000 (62.00%)

the time of this epoch:[45.59594202041626 s]
Train Epoch: 7 

Train Epoch: 12 [25600/50000 (57%)]	Loss: 0.545958, Accuracy: 80.86
Train Epoch: 12 [28160/50000 (62%)]	Loss: 0.557921, Accuracy: 78.91
Train Epoch: 12 [30720/50000 (68%)]	Loss: 0.562885, Accuracy: 80.08
Train Epoch: 12 [33280/50000 (74%)]	Loss: 0.573832, Accuracy: 80.47
Train Epoch: 12 [35840/50000 (80%)]	Loss: 0.565834, Accuracy: 78.91
Train Epoch: 12 [38400/50000 (85%)]	Loss: 0.584349, Accuracy: 80.86
Train Epoch: 12 [40960/50000 (91%)]	Loss: 0.662645, Accuracy: 78.52
Train Epoch: 12 [43520/50000 (97%)]	Loss: 0.601087, Accuracy: 77.34

Validation set: Average loss: 0.7937, Accuracy: 3693/5000 (73.00%)

the time of this epoch:[45.13844347000122 s]
Train Epoch: 13 [0/50000 (0%)]	Loss: 0.642801, Accuracy: 75.78
Train Epoch: 13 [2560/50000 (6%)]	Loss: 0.491489, Accuracy: 81.25
Train Epoch: 13 [5120/50000 (11%)]	Loss: 0.574425, Accuracy: 80.86
Train Epoch: 13 [7680/50000 (17%)]	Loss: 0.621274, Accuracy: 77.34
Train Epoch: 13 [10240/50000 (23%)]	Loss: 0.533319, Accuracy: 82.42
Train Epoch

Train Epoch: 18 [35840/50000 (80%)]	Loss: 0.435257, Accuracy: 83.98
Train Epoch: 18 [38400/50000 (85%)]	Loss: 0.562506, Accuracy: 81.64
Train Epoch: 18 [40960/50000 (91%)]	Loss: 0.451398, Accuracy: 86.33
Train Epoch: 18 [43520/50000 (97%)]	Loss: 0.531987, Accuracy: 80.08

Validation set: Average loss: 0.6196, Accuracy: 3907/5000 (78.00%)

the time of this epoch:[45.64635920524597 s]
Train Epoch: 19 [0/50000 (0%)]	Loss: 0.369795, Accuracy: 85.94
Train Epoch: 19 [2560/50000 (6%)]	Loss: 0.422641, Accuracy: 84.38
Train Epoch: 19 [5120/50000 (11%)]	Loss: 0.453015, Accuracy: 83.98
Train Epoch: 19 [7680/50000 (17%)]	Loss: 0.378072, Accuracy: 85.94
Train Epoch: 19 [10240/50000 (23%)]	Loss: 0.479218, Accuracy: 84.38
Train Epoch: 19 [12800/50000 (28%)]	Loss: 0.446828, Accuracy: 82.42
Train Epoch: 19 [15360/50000 (34%)]	Loss: 0.461119, Accuracy: 85.16
Train Epoch: 19 [17920/50000 (40%)]	Loss: 0.400610, Accuracy: 87.11
Train Epoch: 19 [20480/50000 (45%)]	Loss: 0.383897, Accuracy: 87.50
Train Epoch


Validation set: Average loss: 0.4787, Accuracy: 4249/5000 (84.00%)

the time of this epoch:[45.619932889938354 s]
Train Epoch: 25 [0/50000 (0%)]	Loss: 0.231624, Accuracy: 91.41
Train Epoch: 25 [2560/50000 (6%)]	Loss: 0.362761, Accuracy: 89.84
Train Epoch: 25 [5120/50000 (11%)]	Loss: 0.258635, Accuracy: 91.41
Train Epoch: 25 [7680/50000 (17%)]	Loss: 0.373098, Accuracy: 85.94
Train Epoch: 25 [10240/50000 (23%)]	Loss: 0.315651, Accuracy: 89.84
Train Epoch: 25 [12800/50000 (28%)]	Loss: 0.236082, Accuracy: 91.80
Train Epoch: 25 [15360/50000 (34%)]	Loss: 0.355143, Accuracy: 89.06
Train Epoch: 25 [17920/50000 (40%)]	Loss: 0.208655, Accuracy: 91.80
Train Epoch: 25 [20480/50000 (45%)]	Loss: 0.280318, Accuracy: 90.23
Train Epoch: 25 [23040/50000 (51%)]	Loss: 0.352141, Accuracy: 89.06
Train Epoch: 25 [25600/50000 (57%)]	Loss: 0.398950, Accuracy: 86.33
Train Epoch: 25 [28160/50000 (62%)]	Loss: 0.304791, Accuracy: 90.23
Train Epoch: 25 [30720/50000 (68%)]	Loss: 0.369378, Accuracy: 85.94
Train Epoc

Train Epoch: 31 [5120/50000 (11%)]	Loss: 0.200296, Accuracy: 92.97
Train Epoch: 31 [7680/50000 (17%)]	Loss: 0.259731, Accuracy: 89.84
Train Epoch: 31 [10240/50000 (23%)]	Loss: 0.325326, Accuracy: 89.45
Train Epoch: 31 [12800/50000 (28%)]	Loss: 0.187430, Accuracy: 94.53
Train Epoch: 31 [15360/50000 (34%)]	Loss: 0.244801, Accuracy: 90.23
Train Epoch: 31 [17920/50000 (40%)]	Loss: 0.214703, Accuracy: 91.02
Train Epoch: 31 [20480/50000 (45%)]	Loss: 0.194716, Accuracy: 94.14
Train Epoch: 31 [23040/50000 (51%)]	Loss: 0.180351, Accuracy: 93.36
Train Epoch: 31 [25600/50000 (57%)]	Loss: 0.261332, Accuracy: 89.84
Train Epoch: 31 [28160/50000 (62%)]	Loss: 0.185743, Accuracy: 94.53
Train Epoch: 31 [30720/50000 (68%)]	Loss: 0.221609, Accuracy: 91.41
Train Epoch: 31 [33280/50000 (74%)]	Loss: 0.213749, Accuracy: 91.80
Train Epoch: 31 [35840/50000 (80%)]	Loss: 0.240427, Accuracy: 91.41
Train Epoch: 31 [38400/50000 (85%)]	Loss: 0.245609, Accuracy: 91.80
Train Epoch: 31 [40960/50000 (91%)]	Loss: 0.205142

Train Epoch: 37 [15360/50000 (34%)]	Loss: 0.213533, Accuracy: 91.80
Train Epoch: 37 [17920/50000 (40%)]	Loss: 0.210548, Accuracy: 92.19
Train Epoch: 37 [20480/50000 (45%)]	Loss: 0.253925, Accuracy: 91.80
Train Epoch: 37 [23040/50000 (51%)]	Loss: 0.215963, Accuracy: 92.19
Train Epoch: 37 [25600/50000 (57%)]	Loss: 0.171909, Accuracy: 93.36
Train Epoch: 37 [28160/50000 (62%)]	Loss: 0.224016, Accuracy: 94.92
Train Epoch: 37 [30720/50000 (68%)]	Loss: 0.151392, Accuracy: 94.92
Train Epoch: 37 [33280/50000 (74%)]	Loss: 0.125350, Accuracy: 94.92
Train Epoch: 37 [35840/50000 (80%)]	Loss: 0.191303, Accuracy: 93.36
Train Epoch: 37 [38400/50000 (85%)]	Loss: 0.183572, Accuracy: 94.14
Train Epoch: 37 [40960/50000 (91%)]	Loss: 0.179196, Accuracy: 93.75
Train Epoch: 37 [43520/50000 (97%)]	Loss: 0.193141, Accuracy: 93.75

Validation set: Average loss: 0.4682, Accuracy: 4332/5000 (86.00%)

the time of this epoch:[45.55517935752869 s]
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.169538, Accuracy: 92.19
Train E

Train Epoch: 43 [25600/50000 (57%)]	Loss: 0.087618, Accuracy: 97.27
Train Epoch: 43 [28160/50000 (62%)]	Loss: 0.187053, Accuracy: 93.36
Train Epoch: 43 [30720/50000 (68%)]	Loss: 0.105647, Accuracy: 96.09
Train Epoch: 43 [33280/50000 (74%)]	Loss: 0.169502, Accuracy: 95.31
Train Epoch: 43 [35840/50000 (80%)]	Loss: 0.123194, Accuracy: 95.31
Train Epoch: 43 [38400/50000 (85%)]	Loss: 0.110913, Accuracy: 96.48
Train Epoch: 43 [40960/50000 (91%)]	Loss: 0.150977, Accuracy: 95.70
Train Epoch: 43 [43520/50000 (97%)]	Loss: 0.175949, Accuracy: 93.75

Validation set: Average loss: 0.4792, Accuracy: 4348/5000 (86.00%)

the time of this epoch:[45.52462816238403 s]
Train Epoch: 44 [0/50000 (0%)]	Loss: 0.133412, Accuracy: 93.75
Train Epoch: 44 [2560/50000 (6%)]	Loss: 0.096028, Accuracy: 96.88
Train Epoch: 44 [5120/50000 (11%)]	Loss: 0.098115, Accuracy: 96.09
Train Epoch: 44 [7680/50000 (17%)]	Loss: 0.098042, Accuracy: 96.88
Train Epoch: 44 [10240/50000 (23%)]	Loss: 0.074757, Accuracy: 98.05
Train Epoch

Train Epoch: 49 [35840/50000 (80%)]	Loss: 0.137849, Accuracy: 94.92
Train Epoch: 49 [38400/50000 (85%)]	Loss: 0.109643, Accuracy: 96.88
Train Epoch: 49 [40960/50000 (91%)]	Loss: 0.099273, Accuracy: 96.09
Train Epoch: 49 [43520/50000 (97%)]	Loss: 0.183204, Accuracy: 95.31

Validation set: Average loss: 0.5065, Accuracy: 4334/5000 (86.00%)

the time of this epoch:[45.60560369491577 s]


# Step 3: Test

In [9]:
test(epoch)


Test set: Average loss: 0.5929, Accuracy: 8561/10000 (85.61%)

